In [ ]:
# Lifecycle node states
#
# Get trace data using the provided launch file:
#    $ ros2 launch tracetools_analysis lifecycle_states.launch.py

In [ ]:
path = '~/.ros/tracing/lifecycle-node-state/'

In [ ]:
import sys
# Add paths to tracetools_analysis and tracetools_read.
# There are two options:
#   1. from source, assuming a workspace with:
#       src/tracetools_analysis/
#       src/ros-tracing/ros2_tracing/tracetools_read/
sys.path.insert(0, '../')
sys.path.insert(0, '../../../ros-tracing/ros2_tracing/tracetools_read/')
#   2. from Debian packages, setting the right ROS 2 distro:
#ROS_DISTRO = 'rolling'
#sys.path.insert(0, f'/opt/ros/{ROS_DISTRO}/lib/python3.8/site-packages')
import datetime as dt

from bokeh.palettes import Category10
from bokeh.plotting import figure
from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.layouts import row
from bokeh.models import ColumnDataSource
from bokeh.models import DatetimeTickFormatter
from bokeh.models import PrintfTickFormatter
import numpy as np
import pandas as pd

from tracetools_analysis.loading import load_file
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

In [ ]:
# Process
events = load_file(path)
handler = Ros2Handler.process(events)
#handler.data.print_data()

In [ ]:
data_util = Ros2DataModelUtil(handler.data)

state_intervals = data_util.get_lifecycle_node_state_intervals()
for handle, states in state_intervals.items():
    print(handle)
    print(states.to_string())

output_notebook()
psize = 450

In [ ]:
# Plot
colors = Category10[10]

lifecycle_node_names = {
    handle: data_util.get_lifecycle_node_handle_info(handle)['lifecycle node'] for handle in state_intervals.keys()
}
states_labels = []
start_times = []

fig = figure(
    y_range=list(lifecycle_node_names.values()),
    title='Lifecycle states over time',
    y_axis_label='node',
    plot_width=psize*2, plot_height=psize,
)

for lifecycle_node_handle, states in state_intervals.items():
    lifecycle_node_name = lifecycle_node_names[lifecycle_node_handle]

    start_times.append(states['start_timestamp'].iloc[0])
    for index, row in states.iterrows():
        # TODO fix end
        if index == max(states.index):
            continue
        start = row['start_timestamp']
        end = row['end_timestamp']
        state = row['state']
        if state not in states_labels:
            states_labels.append(state)
        state_index = states_labels.index(state)
        fig.line(
            x=[start, end],
            y=[lifecycle_node_name]*2,
            line_width=10.0,
            line_color=colors[state_index],
            legend_label=state,
        )

fig.title.align = 'center'
fig.xaxis[0].formatter = DatetimeTickFormatter(seconds=['%Ss'])
fig.xaxis[0].axis_label = 'time (' + min(start_times).strftime('%Y-%m-%d %H:%M') + ')'
show(fig)